In [1]:
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
data = Path("../NFL_QB_Clustering/Resources/FantasyPros_Fantasy_Football_Statistics_QB.csv")
df = pd.read_csv(data)
df = df.drop(["Rank", "ROST", "G"], axis = 1)
df.head()

,Player,CMP,ATT,PCT,YDS,Y/A,TD,INT,SACKS,ATT.1,YDS.1,TD.1,FL,FPTS,FPTS/G
0,Josh Allen (BUF),147.0,205.0,71.7,"1,576",7.7,13.0,6.0,9.0,22.0,131.0,3.0,1.0,138.2,23.0
1,Jalen Hurts (PHI),141.0,213.0,66.2,"1,542",7.2,7.0,7.0,14.0,63.0,253.0,5.0,1.0,136.0,22.7
2,Tua Tagovailoa (MIA),140.0,197.0,71.1,"1,876",9.5,14.0,5.0,6.0,14.0,21.0,0.0,1.0,126.2,21.0
3,Patrick Mahomes II (KC),153.0,224.0,68.3,"1,593",7.1,11.0,5.0,6.0,29.0,185.0,0.0,0.0,121.1,20.2
4,Jared Goff (DET),141.0,203.0,69.5,"1,618",8.0,11.0,3.0,10.0,16.0,15.0,2.0,0.0,119.1,19.9


In [3]:
##Scaling the data
df = df.set_index("Player")



In [16]:
df.dtypes
df["YDS"] = df["YDS"].replace({',':''}, regex = True)
df["YDS"] = df["YDS"].astype(float)
print(len(df))
df = df.dropna(how="any")
print(len(df))
players = df.index.values.tolist()
# players[:5]


108
108


['Josh Allen (BUF)',
 'Jalen Hurts (PHI)',
 'Tua Tagovailoa (MIA)',
 'Patrick Mahomes II (KC)',
 'Jared Goff (DET)']

In [14]:
## Create transformation column
column_names = df.columns.tolist()
df_scaled = StandardScaler().fit_transform(df)
df_transformed = pd.DataFrame(df_scaled, columns = column_names)


,CMP,ATT,PCT,YDS,Y/A,TD,INT,SACKS,ATT.1,YDS.1,TD.1,FL,FPTS,FPTS/G
0,1.910928,1.687043,1.250522,1.917772,1.105026,2.823597,2.117926,0.613601,1.160713,1.609842,2.954196,0.682925,2.505010,2.312925
1,1.805128,1.779731,1.085060,1.861824,0.980788,1.246277,2.578159,1.305343,4.436453,3.559945,5.169843,0.682925,2.454723,2.273523
2,1.787495,1.594356,1.232472,2.411431,1.552282,3.086483,1.657692,0.198556,0.521544,-0.148448,-0.369274,0.682925,2.230718,2.050245
3,2.016728,1.907176,1.148236,1.945746,0.955940,2.297823,1.657692,0.198556,1.719986,2.473002,-0.369274,-0.451781,2.114144,1.945173
4,1.805128,1.663871,1.184337,1.986884,1.179568,2.297823,0.737226,0.751949,0.681336,-0.244355,1.846372,-0.451781,2.068429,1.905771


In [6]:
# Find the inertia values
inertia = []
k = list(range(1,11))

In [7]:
for i in k:
    k_model = KMeans(n_clusters=i, random_state = 0)
    k_model.fit(df_transformed)
    inertia.append(k_model.inertia_)


c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto

In [8]:
elbow_df = pd.DataFrame({"k": k, "inertia": inertia})
elbow_df.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k
)

:Curve   [k]   (inertia)

In [11]:
## Create the new model with dictated elbow curve
QB_model = KMeans(n_clusters=2 ,random_state = 42)
QB_model.fit(df_transformed)
predictions = QB_model.predict(df_transformed)

c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [22]:
df_QB = df_transformed.copy()
df_QB["QB_segementation"] = predictions
df_QB["QB"] = players
df_QB = df_QB.set_index("QB")
df_QB.head()


,CMP,ATT,PCT,YDS,Y/A,TD,INT,SACKS,ATT.1,YDS.1,TD.1,FL,FPTS,FPTS/G,QB_segementation
QB,,,,,,,,,,,,,,,
Josh Allen (BUF),1.910928,1.687043,1.250522,1.917772,1.105026,2.823597,2.117926,0.613601,1.160713,1.609842,2.954196,0.682925,2.505010,2.312925,1
Jalen Hurts (PHI),1.805128,1.779731,1.085060,1.861824,0.980788,1.246277,2.578159,1.305343,4.436453,3.559945,5.169843,0.682925,2.454723,2.273523,1
Tua Tagovailoa (MIA),1.787495,1.594356,1.232472,2.411431,1.552282,3.086483,1.657692,0.198556,0.521544,-0.148448,-0.369274,0.682925,2.230718,2.050245,1
Patrick Mahomes II (KC),2.016728,1.907176,1.148236,1.945746,0.955940,2.297823,1.657692,0.198556,1.719986,2.473002,-0.369274,-0.451781,2.114144,1.945173,1
Jared Goff (DET),1.805128,1.663871,1.184337,1.986884,1.179568,2.297823,0.737226,0.751949,0.681336,-0.244355,1.846372,-0.451781,2.068429,1.905771,1


In [25]:
df_QB.hvplot.scatter(
    x="TD",
    y="YDS",
    by="QB_segementation",
    hover_cols="QB",
    title="QB CMP-ATT"
)

:NdOverlay   [QB_segementation]
   :Scatter   [TD]   (YDS,QB)